In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from utils_nlp import preprocess_text

ModuleNotFoundError: No module named 'utils_nlp'

In [ ]:
df = pd.read_csv("data/bank_reviews_with_sentiment.csv")
df["clean_review"] = df["review"].fillna("").apply(preprocess_text)


In [ ]:
tfidf = TfidfVectorizer(max_features=150, ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(df["clean_review"])
feature_names = tfidf.get_feature_names_out()
keywords = []
for row in tfidf_matrix:
    top_indices = row.toarray()[0].argsort()[-5:][::-1]
    kws = [feature_names[idx] for idx in top_indices if row[0, idx] > 0]
    keywords.append(", ".join(kws))
df["keywords"] = keywords

# Manual/grouped themes (you will document this logic in README/notebook)
def assign_theme(keywords):
    if any(word in keywords for word in ["login", "access", "password"]):
        return "Account Access Issues"
    if any(word in keywords for word in ["transfer", "transaction", "delay", "balance"]):
        return "Transaction Performance"
    if any(word in keywords for word in ["ui", "interface", "easy", "design"]):
        return "User Interface & Experience"
    if any(word in keywords for word in ["support", "help", "response"]):
        return "Customer Support"
    if any(word in keywords for word in ["feature", "add", "missing"]):
        return "Feature Requests"
    return "Other"

df["theme"] = df["keywords"].apply(assign_theme)
df.to_csv("data/bank_reviews_with_themes.csv", index=False)